In [1]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import os
import cv2
from time import time
import numpy as np
import pandas as pd
from yolov5.models.yolo import Model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torchvision


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DL Project/TrafficSign DataSet.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def ResizeImage(image: np.ndarray, newWidth: int, newHeight: int) -> tuple:
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    return resizedImage

In [5]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/tafficSignAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=", ")
    print(annotationsDataFrame.columns)
    uniqueSigns = annotationsDataFrame['Catoregy'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        # print("."+row[1])
        image = cv2.imread("."+row[1])
        # print(image.shape)
        images.append(image)
        annotations.append(uniqueSigns.index(row[0]))
    del annotationsDataFrame
    return images, annotations, len(uniqueSigns)


In [6]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        label = annotations[i]
        resizedImage = ResizeImage(
            image, resize[0], resize[1])
        resizedImages.append(resizedImage)
        newAnnotations.append(label)

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [7]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label)
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [8]:
def TrainModel(model, dataLoader, epochs, optimizer, criterion, device):
    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch+1, epochs))
        startTime = time()
        runningLoss = 0
        dataLoaderLen = len(dataLoader)
        runningAccuracy = 0

        for i, (inputs, labels) in enumerate(dataLoader):
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            runningLoss += loss.item() * inputs.size(0)
            runningAccuracy += torch.sum(preds == labels.data)
            if(((i*100)//dataLoaderLen) % 10 == 0):
                print((i*100//dataLoaderLen), end="%,")
        endTime = time()
        timeTaken = endTime-startTime
        epochLoss = runningLoss / dataLoaderLen
        epochAccuracy = runningAccuracy.double() / dataLoaderLen
        print()
        print("Training Loss: {:.4f}".format(epochLoss))
        print("Training Accuracy: {:.4f}".format(epochAccuracy))
        print("Time taken: {}min, {}, secs".format(timeTaken//60, timeTaken % 60))


    print('Training complete')
    return model


In [9]:
batchSize = 32
inputShape = (416, 416)
epochs = 100
numAnchors = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
print("Using {} device".format(device))


Using cuda device


In [11]:
images, annotations, numClasses = LoadDataSet("./TrafficSign DataSet")


Index(['Catoregy', 'path'], dtype='object')


<ipython-input-5-acfaf00baccb>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=", ")


In [12]:
X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations


In [13]:
trainDataLoader, valDataLoader = CreateDataLoaders(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val


In [14]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, numClasses)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [15]:
trainedModel = TrainModel(model, trainDataLoader, epochs, optimizer, criterion, device)

Epoch 1/100:
0%,0%,10%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 131.3604
Training Accuracy: 5.9667
Time taken: 2.0min, 17.332876443862915, secs
Epoch 2/100:
0%,0%,10%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 101.0120
Training Accuracy: 10.2000
Time taken: 2.0min, 15.64116096496582, secs
Epoch 3/100:
0%,0%,10%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 79.1213
Training Accuracy: 13.7250
Time taken: 2.0min, 15.636786460876465, secs
Epoch 4/100:
0%,0%,10%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 62.2353
Training Accuracy: 17.2833
Time taken: 2.0min, 15.70071816444397, secs
Epoch 5/100:
0%,0%,10%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,60%,70%,70%,80%,80%,90%,90%,
Training Loss: 49.2693
Training Accuracy: 19.8167
Time taken: 2.0min, 15.70964002609253, secs
Epoch 6/100:
0%,0%,10%,10%,20%,20%,30%,30%,40%,40%,50%,50%,60%,60%,

In [16]:
torch.save(trainedModel.state_dict(), 'signPredictionV1.pth')
trainedModel.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:

# Load the model for inference
model.load_state_dict(torch.load('signPredictionV1.pth'))
model.eval()

# Test the model on a single image
from PIL import Image
import numpy as np

def preprocess_image(image_path):
  img = Image.open(image_path)
  transform = data_transforms['val']
  img_tensor = transform(img).unsqueeze(0)
  return img_tensor

def predict_class(image_path):
  img_tensor = preprocess_image(image_path)
  img_tensor = img_tensor.to(device)

  with torch.no_grad():
      output = model(img_tensor)
      _, predicted_class = torch.max(output, 1)

  return predicted_class.item()


In [ ]:
print(predict_class)